[Chapter 2] Building an LLM - Data Preparation & Sampling

In [ ]:
#%pip install --upgrade torch
#import torch
#torch.__version__

'2.7.1+cpu'

In [2]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x14ee03f5df0>)

In [3]:
with open("the-verdict.txt","r",encoding='utf-8') as f:
    raw_text = f.read()

print("total number of character:", len(raw_text))
print(raw_text[:99])

total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


from text to tokenized text

In [4]:
import re
text = "Hello, world. Is this--a test?"
# split text by whitespace(\s), comma(,) and periods(.)
result = re.split(r'([,.:;?_!"()\']|--|\s)', text) 
# remove redundant whitespace characters
# however, keeping whitespace can be useful if we train models that are sensitive to the exact structure of the text
result = [item for item in result if item.strip()] 
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [5]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


converting tokens into token IDs

In [6]:
# build vocabulary
# list of all unique tokens and sort them alphabetically
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)
# create vocabulary
vocab = {token: integer for integer, token in enumerate(all_words)}
# print first 51 entries for illustration
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


construct a complete text tokenizer

In [7]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab # map token to token id
        self.int_to_str = {i: s for s, i in vocab.items()} # map token id back to text token
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text) # remove spaces before the punctuation
        return text

In [8]:
# test tokenizer: encode (from text to ids)
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know, "
        Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [9]:
# test tokenizer: decode (from ids to text)
print(tokenizer.decode(ids))

It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [32]:
# test tokenizer: text not in training set
# the error highlights the need to consider large and diverse training sets to extend the vocab when working on LLMs
text = "Hello, do you like tea?"
#print(tokenizer.encode(text))

add special context tokens

In [11]:
all_tokens = sorted(list(set(preprocessed)))
# <|endoftext|> to seperate two unrelated text sources
# <|unk|> to represent new and unknown words that were not part of the training set
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

1132
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [12]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab # map token to token id
        self.int_to_str = {i: s for s, i in vocab.items()} # map token id back to text token
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # replace unknown text token with "<|unk|>"
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text) # remove spaces before the punctuation
        return text

In [13]:
# test text
text1 = "Hello, do you like tea?"
text2 = "In the sulit terraces of the palace."
text = " <|endoftext|> ".join([text1, text2])
print(text)

Hello, do you like tea? <|endoftext|> In the sulit terraces of the palace.


In [14]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 1131, 984, 722, 988, 1131, 7]


In [15]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the <|unk|> terraces of the <|unk|>.


byte pair encoding (BPE)

In [16]:
from importlib.metadata import version
import tiktoken
print("tiktokn version:", version("tiktoken"))

tiktokn version: 0.9.0


In [17]:
tokenizer = tiktoken.get_encoding("gpt2")

In [18]:
# from text to token ids
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"}) 
#<|endoftext|> gets id=50256 since vocab_size=50257
print(integers)

#BPE breaks down unknown words into subwords and individual characters.
#BPE tokenizer then can parse any word and doesn't need to replace unknown with <|unk|>
print(tokenizer.encode("Akwirw ier."))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 33154, 270, 8812, 2114, 286, 262, 20562, 13]
[33901, 86, 343, 86, 220, 959, 13]


In [19]:
# from token ids to text
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sulit terraces of the palace.


data sampling with a sliding window

In [20]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [21]:
# input-target pairs for training an LLM
context_size = 4 # number of tokens inlcuded in the input
enc_sample = enc_text[50:]

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))


[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257
 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [22]:
import torch
from torch.utils.data import Dataset, DataLoader

# data loader that iterates over the input dataset and returns the inputs and targets as PyTorch tensors
class GPTDatasetV1(Dataset):

    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        
        for i in range(0, len(token_ids) - max_length, stride): # stride indicates the number of positions in the inputs shift across batches
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1] # corresponding next-word prediction targets by shifiting the input by one positions
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    # return total number of rows in the dataset
    def __len__(self):
        return len(self.input_ids)
    
    # return a single row from the dataset
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [23]:
# data loader to generate batches with input-with pairs

def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last, # drop the last batch if it is shorter than the specified batch_size to prevent loss spikes during training
        num_workers = num_workers # the number of cpu processes to use for preprocessing
    )

    return dataloader

In [24]:
# test: batch_size = 1, max_len = 4, stride = 1
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

# convert dataloader into a python iterator to fetch the next entry via python's next() function
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [25]:
# test: batch_size = 8, max_len = 4, stride = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("inputs:\n", inputs)
print("\ntargets:\n", targets)

inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


create token embeddings

In [26]:
# hands-on example
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6 #token size in vocabulary
output_dim = 3 #embedding dimensions
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

# vocab_size(6) x output_dim(3) weight matrix 
# these small, random weight values are optimized during LLM training/optimization
print(embedding_layer.weight) 
print("\n")

# embedding vector of a token ID
print(embedding_layer(torch.tensor([3])))
print("\n")

# embedding vector of input ids
print(embedding_layer(input_ids))

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


token embeddings + positional embeddings = input embeddings

In [27]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [28]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("token ids:\n", inputs)
print("\ninputs shape:\n", inputs.shape)

token ids:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

inputs shape:
 torch.Size([8, 4])


In [29]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [30]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [31]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
